In SimPy, the behaviors of active components such as customers or vehicles are modeled with *processes*. These processes live in an *environment*. They interact with the environment and with each other via *events*.

In [11]:
import simpy
from random import random, seed

## Simulate Waiters

In [4]:
def waiter(env):
    while True: # Simulate until the time limit
        print(f"Start taking orders from customers at {env.now}")
        take_order_duration = 5
        yield env.timeout(take_order_duration) # models duration

        print(f'Start giving the orders to the cooks at {env.now}')
        give_order_duration = 2
        yield env.timeout(give_order_duration)

        print(f'Start serving customers food at {env.now}\n')
        serve_order_duration = 5
        yield env.timeout(serve_order_duration)

In [2]:
env = simpy.Environment() # the environment where the waiter lives
env.process(waiter(env)) # pass the waiter to the environment
env.run(until=30) # Run simulation until 30s

Start taking orders from customers at 0
Start giving the orders to the cooks at 5
Start serving customers food at 7

Start taking orders from customers at 12
Start giving the orders to the cooks at 17
Start serving customers food at 19

Start taking orders from customers at 24
Start giving the orders to the cooks at 29


## Simulate Customers

In [7]:
def customer(env, name, restaurant, **duration):
    while True:
        yield env.timeout(random()*10) # There is a new customer between 0 and 10 minutes
        print(f"{name} enters the restaurant and for the waiter to come at {round(env.now, 2)}")
        with restaurant.request() as req:
            yield req 

            print(f"Seats are available. {name} get seated at {round(env.now, 2)}")
            yield env.timeout(duration['get_sitted'])

            print(f"{name} starts looking at the menu at {round(env.now, 2)}")
            yield env.timeout(duration['choose_food'])

            print(f'Waiters start getting the order from {name} at {round(env.now, 2)}')
            yield env.timeout(duration['give_order'])

            print(f'{name} starts waiting for food at {round(env.now, 2)}')
            yield env.timeout(duration['wait_for_food'])

            print(f'{name} starts eating at {round(env.now, 2)}')
            yield env.timeout(duration['eat'])

            print(f'{name} starts paying at {round(env.now, 2)}')
            yield env.timeout(duration['pay'])

            print(f'{name} leaves at {round(env.now, 2)}')

In [9]:
seed(1)
env = simpy.Environment()

# Model restaurant that can only allow 2 customers at once
restaurant = simpy.Resource(env, capacity=2)

In [10]:
durations = {'get_sitted': 1, 'choose_food': 10, 'give_order': 5, 'wait_for_food': 20, 'eat': 45, 'pay': 10}

for i in range(5):
    env.process(customer(env, f'Customer {i}', restaurant, **durations))

env.run(until=95)

Customer 0 enters the restaurant and for the waiter to come at 1.34
Seats are available. Customer 0 get seated at 1.34
Customer 0 starts looking at the menu at 2.34
Customer 3 enters the restaurant and for the waiter to come at 2.55
Seats are available. Customer 3 get seated at 2.55
Customer 3 starts looking at the menu at 3.55
Customer 4 enters the restaurant and for the waiter to come at 4.95
Customer 2 enters the restaurant and for the waiter to come at 7.64
Customer 1 enters the restaurant and for the waiter to come at 8.47
Waiters start getting the order from Customer 0 at 12.34
Waiters start getting the order from Customer 3 at 13.55
Customer 0 starts waiting for food at 17.34
Customer 3 starts waiting for food at 18.55
Customer 0 starts eating at 37.34
Customer 3 starts eating at 38.55
Customer 0 starts paying at 82.34
Customer 3 starts paying at 83.55
Customer 0 leaves at 92.34
Seats are available. Customer 4 get seated at 92.34
Customer 4 starts looking at the menu at 93.34
Cu

## Simulate a Takeout Restaurant with a Limited Supply of Food

We will simulate a restaurant that:

* Only takes take-out orders
* There is only one staff member
* The customers are waiting in line to order food on the menu. The more items a customer orders, the longer the customer needs to wait.

The attributes of this restaurant are:

* `staff` : a resource with the capacity 1 (the staff can only serve one customer at a time)
* `foods` : options of food the restaurant offers
* `available` : number of items per option of food
* `run_out` : events when each option of food runs out
* `when_run_out` : the time when each option of food runs out
* `rejected_customers` : number of customers who leave the line because their food option runs out

In [17]:
from collections import namedtuple

NUM_ITEMS = 10 # Number of items per food option

staff = simpy.Resource(env, capacity=1)
foods = ['Spicy Chicken', 'Poached Chicken', 'Tomato Chicken Skillet', 
         'Honey Mustard Chicken']
available = {food: NUM_ITEMS for food in foods} 
run_out = {food: env.event() for food in foods}
when_run_out = {food: None for food in foods}
rejected_customers = {food: 0 for food in foods}

Restaurant = namedtuple('Restaurant', 'staff, foods, available,'
                        'run_out, when_run_out, rejected_customers')
restaurant = Restaurant(staff, foods, available, run_out,
                        when_run_out, rejected_customers)

We also want to create a customer who is waiting in line. A customer will:

* Leave if there is not enough food left
* Order food if there is enough food left

If there is no food left after the customer orders a particular kind of food, we will declare that the chosen food has run out.

Two new concepts in the code below are:

* `restaurant.staff.request` : Request usage of the resource. In this case, the customer requests a service from the staff member.
* `restaurant.run_out[food].succeed` : run_out is an event that can either succeed or fail. 

In this case, if the event run_out succeeds, it signals that the restaurant runs out of a food option.

In [18]:
def customer(env, food, num_food_order, restaurant):
    """Customer tries to order a certain number of a particular food, 
    if that food ran out, customer leaves. If there is enough food left,
    customer orders food."""

    with restaurant.staff.request() as customer:

        # If there is not enough food left, customer leaves
        if restaurant.available[food] < num_food_order:
            restaurant.rejected_customers[food] +=1
            return

        # If there is enough food left, customer orders food
        restaurant.available[food] -= num_food_order
        # The time it takes to prepare food
        yield env.timeout(10*num_food_order)

        # If there is no food left after customer orders, trigger run out event
        if restaurant.available[food] == 0:
            restaurant.run_out[food].succeed()
            restaurant.when_run_out[food] = env.now

        yield env.timeout(2)

Run the simulation:

In [19]:
import random 

RANDOM_SEED = 20
SIM_TIME  = 240

random.seed(RANDOM_SEED)
env = simpy.Environment()

# Start process and run
env.process(customer_arrivals(env, restaurant))
env.run(until=SIM_TIME)

for food in foods:
    if restaurant.run_out[food]:
        print(f'The {food} ran out {round(restaurant.when_run_out[food], 2)} '
            'minutes after the restaurant opens.')
            
        print(f'Number of people leaving queue when the {food} ran out is ' 
        f'{restaurant.rejected_customers[food]}.\n')

NameError: name 'customer_arrivals' is not defined